In [18]:
# neatness of code:
%load_ext pycodestyle_magic
%pycodestyle_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [1]:
# Packages:
import sqlite3
import pandas as pd
import numpy as np
# First que list all objects in the database:
conn = sqlite3.connect('/home/ferran/Desktop/master/R&I/ml/final_project_skeleton/db.sqlite3')
pd.read_sql(""" SELECT * FROM sqlite_master;""", conn)
# load tables:
long = pd.read_sql("""SELECT * FROM CountryIndicators;""", conn)
# Not countries
exclude_list = ['Arab World', 'Caribbean small states', 'Central Europe and the Baltics',
 'East Asia & Pacific \(all income levels',
 'East Asia & Pacific \(developing only', 'Euro area',
 'Europe & Central Asia \(all income levels',
 'Europe & Central Asia \(developing only', 'European Union',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries \(HIPC', 'High income',
 'High income: nonOECD', 'High income: OECD',
 'Latin America & Caribbean \(all income levels',
 'Latin America & Caribbean \(developing only',
 'Least developed countries: UN classification', 'Low & middle income',
 'Low income', 'Lower middle income',
 'Middle East & North Africa \(all income levels',
 'Middle East & North Africa \(developing only', 'Middle income',
 'North America' 'OECD members' ,'Other small states',
 'Pacific island small states', 'Small states', 'South Asia',
 'Sub-Saharan Africa \(all income levels',
 'Sub-Saharan Africa \(developing only' ,'Upper middle income' ,'World', 'North America', 'OECD members']
# LongName to CountryName
country_names = pd.read_sql("""SELECT * FROM Countries;""", conn)
not_country = country_names.loc[country_names["LongName"].isin(exclude_list)]["CountryCode"]
long= long.loc[~long["CountryCode"].isin(not_country)]


In [ ]:
READ DATABASE

In [2]:
df = long.pivot_table(index=['CountryCode','Year'], columns='IndicatorCode',values='Value',aggfunc=np.sum)
df['Country'] = df.index.get_level_values(0)
df.columns = df.columns.str.replace(".", "_")


In [37]:
df["NY_GDP_MKTP_KD_ZG"]
pd.unique(df[df.index.get_level_values(1) == 1960]["NY_GDP_MKTP_KD_ZG"])


array([nan])

In [24]:
len(pd.unique(long["CountryCode"]))
len(pd.unique(df['Country']))
len(pd.unique(df.index.get_level_values(1)))
df.shape # before LM: (11526, 1329)
df.shape # after LM:  (7967, 1332)

(11526, 1329)

LINEAR MIXED MODEL

eliminamos años donde la respuesta es NA!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [3]:
import statsmodels.formula.api as smf

# Creation of covariables for the linear model.
df['lag1']=df['NY_GDP_MKTP_KD_ZG'].shift(1)
df['Time'] = df.index.get_level_values(1)
df = df.dropna(subset=["NY_GDP_MKTP_KD_ZG","lag1"])
# Countries strings to numeric values:
groups = df[["Country"]].replace(pd.unique(df.Country), 
            list(range(0,len(pd.unique(df.Country)))))
groups = pd.to_numeric(groups.Country)
# mixed linear model with group as random effect.
df_sub = df[["NY_GDP_MKTP_KD_ZG","Country","lag1"]]
md = smf.mixedlm("NY_GDP_MKTP_KD_ZG ~ lag1", df_sub, groups=groups)#re_formula="~Time",groups=groups,  #df["Country"].to_numpy()
mod = md.fit()
print(mod.summary())
df['residuals'] = mod.resid
df['Country'] = groups
mod.summary()
df.shape

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: NY_GDP_MKTP_KD_ZG
No. Observations: 7967    Method:             REML             
No. Groups:       216     Scale:              39.7507          
Min. group size:  1       Log-Likelihood:     -26053.5410      
Max. group size:  49      Converged:          Yes              
Mean group size:  36.9                                         
-----------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|   [0.025   0.975]
-----------------------------------------------------------------
Intercept     2.908      0.113   25.829   0.000    2.687    3.128
lag1          0.277      0.011   24.737   0.000    0.255    0.299
Group Var     1.128      0.038                                   



(7967, 1332)

FEATURE SELECTION
But in short, the mutual information between two variables, measures how much a given feature can explain another (target), or more technically, how much information about the target will variable will be obtained by having observed a feature.

The mutual information between two random variables is a non-negative value that measures the dependencies between variables. The mutual information between independent random variables is equal to 0, and the larger the mutual information, the stronger the dependency.
This function relies on the nonparametric method of entropy estimation based on k-nearest neighbor distance


In [56]:
from sklearn.feature_selection import f_regression, mutual_info_regression

# Filter/impute vars with NA
df_fewNA = df[df.columns[(df.isnull().sum(axis = 0)/df.shape[0]<=0.3)]] # 0.3:210 vars
country2=df_fewNA['Country']
df_fewNA = df_fewNA.groupby(country2).transform(lambda x: x.fillna(x.ffill().bfill()))
df_fewNA = df_fewNA.fillna(df.mean())
df_fewNA["Country"] = country2
# Feature selection
covs = df_fewNA.drop(["NY_GDP_MKTP_KD_ZG","residuals"], 1)
Y = df_fewNA[['residuals']]
info = mutual_info_regression(covs, np.ravel(Y))
df_varimp =pd.DataFrame(data={'name': covs.columns, 'varimp': info})
# Keep top50
selected_variables = df_varimp.sort_values(by="varimp",ascending=False)[0:49]
selected_variables['name'] = selected_variables['name'].str.replace('_','.')



,name,varimp,0
0,NY.GDP.PCAP.KD.ZG,0.959658,NaN
1,NV.IND.TOTL.KD.ZG,0.258467,NaN
2,NV.SRV.TETC.KD.ZG,0.220456,NaN
3,NV.AGR.TOTL.KD.ZG,0.089266,NaN
4,lag1,0.078674,NaN
5,Time,0.073087,NaN
6,DC.DAC.TOTL.CD,0.062970,NaN
7,DC.DAC.JPNL.CD,0.057926,NaN
8,DT.ODA.ALLD.CD,0.056020,NaN
9,TM.VAL.MRCH.CD.WT,0.053887,NaN


In [ ]:
FETAURE SELECTION: NAMES OF FINAL VARIABLES

In [61]:
# Definition of variables: sql: Indicators
conn = sqlite3.connect('/home/ferran/Desktop/master/R&I/ml/final_project_skeleton/db.sqlite3')
list_of_entries_to_retrive = (selected_variables['name']).tolist()
queryString = 'SELECT IndicatorCode,LongDefinition FROM Indicators WHERE IndicatorCode IN (\'{}\');'.format('\',\''.join([_ for _ in list_of_entries_to_retrive]))
vars_definition = pd.read_sql(queryString, con=conn)
vars_definition = vars_definition.append(pd.DataFrame(data={'IndicatorCode': ["lag1","Country","Time"],
                                    'LongDefinition': ["lag1","Country","Time"]}), ignore_index=True)
vars_definition = vars_definition.sort_values(by="IndicatorCode")
#vars_definition.columns = ['IndicatorCode','name']
# bind cols
selected_variables = selected_variables.sort_values(by="name")
if selected_variables['name'].tolist() == vars_definition['IndicatorCode'].tolist():
    selected_variables["LongDefinition"] = vars_definition['LongDefinition'].tolist()
#df_c = pd.concat([selected_variables.reset_index(drop=True), vars_definition['LongDefinition'].reset_index(drop=True) ], axis=1)




0        AG.LND.AGRI.K2
1        AG.LND.AGRI.ZS
2     AG.LND.ARBL.HA.PC
3        AG.PRD.CREL.MT
4        AG.YLD.CREL.KG
5               Country
6        DC.DAC.DEUL.CD
7        DC.DAC.JPNL.CD
8        DC.DAC.TOTL.CD
9        DT.ODA.ALLD.CD
10       DT.ODA.ALLD.KD
11       DT.ODA.ODAT.CD
12    EN.ATM.CO2E.GF.KT
13    EN.ATM.CO2E.LF.KT
14    EN.ATM.CO2E.SF.KT
15          EN.URB.LCTY
16          IS.AIR.PSGR
17          IT.MLT.MAIN
18       NE.CON.GOVT.CD
19       NE.CON.TETC.CD
20       NE.DAB.TOTL.CD
21       NE.GDI.TOTL.CD
22       NE.IMP.GNFS.CD
23       NV.AGR.TOTL.CD
24       NV.AGR.TOTL.KD
25    NV.AGR.TOTL.KD.ZG
26       NV.IND.TOTL.KD
27    NV.IND.TOTL.KD.ZG
28       NV.IND.TOTL.ZS
29    NV.SRV.TETC.KD.ZG
30       NY.ADJ.DKAP.CD
31       NY.ADJ.NNTY.CD
32    NY.ADJ.NNTY.PC.CD
33    NY.GDP.PCAP.KD.ZG
34       SP.DYN.CBRT.IN
35    SP.DYN.LE00.FE.IN
36       SP.DYN.LE00.IN
37    SP.DYN.LE00.MA.IN
38    SP.DYN.TO65.FE.ZS
39    SP.POP.65UP.TO.ZS
40          SP.POP.TOTL
41          SP.R

READ BDD FOR FINAL MODEL: SQL:CountryIndicators read only important variables

In [74]:
selected_variables= pd.read_csv('selected_variables.txt')
conn = sqlite3.connect('/home/ferran/Desktop/master/R&I/ml/final_project_skeleton/db.sqlite3')

selected_variables = selected_variables['name'].append(pd.Series("NY.GDP.MKTP.KD.ZG"),ignore_index=True).tolist()
queryString = 'SELECT * FROM CountryIndicators WHERE IndicatorCode IN (\'{}\');'.format('\',\''.join([_ for _ in selected_variables]))
vars1 = pd.read_sql(queryString, con=conn)

exclude_list = ['Arab World', 'Caribbean small states', 'Central Europe and the Baltics',
 'East Asia & Pacific \(all income levels',
 'East Asia & Pacific \(developing only', 'Euro area',
 'Europe & Central Asia \(all income levels',
 'Europe & Central Asia \(developing only', 'European Union',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries \(HIPC', 'High income',
 'High income: nonOECD', 'High income: OECD',
 'Latin America & Caribbean \(all income levels',
 'Latin America & Caribbean \(developing only',
 'Least developed countries: UN classification', 'Low & middle income',
 'Low income', 'Lower middle income',
 'Middle East & North Africa \(all income levels',
 'Middle East & North Africa \(developing only', 'Middle income',
 'North America' 'OECD members' ,'Other small states',
 'Pacific island small states', 'Small states', 'South Asia',
 'Sub-Saharan Africa \(all income levels',
 'Sub-Saharan Africa \(developing only' ,'Upper middle income' ,'World', 'North America', 'OECD members']
# LongName to CountryName
country_names = pd.read_sql("""SELECT LongName,CountryCode FROM Countries;""", conn)
not_country = country_names.loc[country_names["LongName"].isin(exclude_list)]["CountryCode"]
vars1= vars1.loc[~long["CountryCode"].isin(not_country)]

vars2 = vars1.pivot_table(index=['CountryCode','Year'], columns='IndicatorCode',values='Value',aggfunc=np.sum)
vars2['Country'] = vars2.index.get_level_values(0)
vars2['Time'] = vars2.index.get_level_values(1)
#vars2.columns = vars2.columns.str.replace(".", "_")

Index(['AG.LND.AGRI.K2', 'AG.LND.AGRI.ZS', 'AG.LND.ARBL.HA.PC',
       'AG.SRF.TOTL.K2', 'AG.YLD.CREL.KG', 'DC.DAC.DEUL.CD', 'DC.DAC.JPNL.CD',
       'DC.DAC.TOTL.CD', 'DT.ODA.ALLD.CD', 'DT.ODA.ALLD.KD', 'DT.ODA.ODAT.CD',
       'EN.ATM.CO2E.GF.KT', 'EN.ATM.CO2E.LF.KT', 'EN.ATM.CO2E.SF.KT',
       'EN.URB.LCTY', 'IS.AIR.PSGR', 'IT.MLT.MAIN', 'NE.CON.GOVT.CD',
       'NE.CON.TETC.CD', 'NE.DAB.TOTL.CD', 'NE.GDI.TOTL.CD', 'NE.IMP.GNFS.CD',
       'NV.AGR.TOTL.CD', 'NV.AGR.TOTL.KD', 'NV.AGR.TOTL.KD.ZG',
       'NV.IND.TOTL.KD', 'NV.IND.TOTL.KD.ZG', 'NV.IND.TOTL.ZS',
       'NV.SRV.TETC.KD.ZG', 'NY.ADJ.DKAP.CD', 'NY.ADJ.NNTY.CD',
       'NY.ADJ.NNTY.PC.CD', 'NY.GDP.MKTP.KD.ZG', 'NY.GDP.PCAP.KD.ZG',
       'SP.DYN.CBRT.IN', 'SP.DYN.LE00.FE.IN', 'SP.DYN.LE00.IN',
       'SP.DYN.LE00.MA.IN', 'SP.DYN.TO65.FE.ZS', 'SP.POP.65UP.TO.ZS',
       'SP.POP.TOTL', 'SP.RUR.TOTL', 'SP.URB.GROW', 'TM.VAL.MRCH.CD.WT',
       'TM.VAL.MRCH.WL.CD', 'TX.VAL.MRCH.CD.WT', 'TX.VAL.MRCH.WL.CD'],
      dtype='object

In [68]:
selected_variables= pd.read_csv('selected_variables.txt')
selected_variables = selected_variables['name'].append(pd.Series("NY_GDP_MKTP_KD_ZG"),ignore_index=True).tolist()
list_of_entries_to_retrive = selected_variables #(selected_variables['name']).tolist()

#sum(selected_variables['name'] == 'NY_GDP_MKTP_KD_ZG')
#selected_variables['name'].append(pd.Series("NY_GDP_MKTP_KD_ZG"),ignore_index=True)
type(selected_variables)

pandas.core.series.Series

In [7]:
from utils import read_database


# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(0, '/home/ferran/Desktop/master/"R&I"/ml/final_project_skeleton/gdp_growth_forecast/utils/')

#import read_database.py

ModuleNotFoundError: No module named 'utils'

In [6]:
#### 1) We find Y: EstimatedGDPGrowth for the next year:
#summ_vars2 = summ_vars[summ_vars['IndicatorName'].str.contains("GDP")]
#summ_vars2[~summ_vars2.IndicatorName.str.contains("% of GDP")]
    # Response variable: GDP growth (annual %): NY.GDP.MKTP.KD.ZG
    # GPD covariate: GDP (constant 2005 US$), NY.GDP.MKTP.KD [look description of response if any doubt].
#long.CountryCode.unique()[1:10]

ERROR! Session/line number was not unique in database. History logging moved to new session 41
7.18.1


1:1: E266 too many leading '#' for block comment
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
4:5: E116 unexpected indentation (comment)
5:5: E116 unexpected indentation (comment)
5:80: E501 line too long (105 > 79 characters)
6:1: E265 block comment should start with '# '
7:11: E225 missing whitespace around operator
